In [ ]:
import subprocess
import os
import threading
import requests
from zipfile import ZipFile
import re
import time
import tarfile

# Step 1: Download the prebuilt UI zip file quietly
zip_url = "https://disk.motoemotovps.xyz/d/Chomikuj/remote-dl-files/archive_name.tar.zip?sign=tZt3Lg3oxxHQ1R2cBOkxx7ORFX1srLY7c8v_9mqtfzI=:0"

print("Downloading prebuilt UI...")
response = requests.get(zip_url, stream=True)
content_disposition = response.headers.get('Content-Disposition')
if content_disposition:
    filename = re.findall("filename=(.+)", content_disposition)[0].strip('"')
else:
    filename = zip_url.split("/")[-1]

zip_file_path = filename

with open(zip_file_path, 'wb') as file:
    for chunk in response.iter_content(chunk_size=8192):
        file.write(chunk)

# Step 2: Create 'metube' folder if it doesn't exist
metube_folder = "metube"
if not os.path.exists(metube_folder):
    os.makedirs(metube_folder)

# Step 3: Unzip the downloaded file quietly into the 'metube' folder
print("Unzipping prebuilt UI...")
with ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(metube_folder)

# Clean up the zip file to save space
os.remove(zip_file_path)

# Step 3: Install Python 3.11 (if necessary)
!sudo apt-get update -qq
!sudo apt-get install python3.11 python3.11-dev python3.11-venv -y -qq

# Set Python 3.11 as the default (automatically)
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 1
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.11 2
!sudo update-alternatives --set python3 /usr/bin/python3.11  # Automatically set Python 3.11 as default

# Install pip for Python 3.11
!sudo apt-get install python3.11-distutils -y -qq
!curl -sS https://bootstrap.pypa.io/get-pip.py | python3.11

# Verify that Python 3.11 is installed and set up correctly
!python3 --version
!pip --version

# Step 4: Install pipenv using Python 3.11
!pip install pipenv -q

# Step 5: Navigate to the base directory
%cd metube

# Step 6: Install all Python dependencies from the Pipfile using Python 3.11
!pipenv --python /usr/bin/python3.11
!pipenv install

# Step 7: Download and prepare bore for tunneling
bore_url = "https://github.com/ekzhang/bore/releases/download/v0.5.1/bore-v0.5.1-x86_64-unknown-linux-musl.tar.gz"
bore_tar_path = "bore.tar.gz"

print("Downloading bore...")
response = requests.get(bore_url)
with open(bore_tar_path, 'wb') as file:
    file.write(response.content)

print("Extracting bore...")
with tarfile.open(bore_tar_path, 'r:gz') as tar:
    tar.extractall()

os.remove(bore_tar_path)
os.chmod("./bore", 0o755)

# Step 8: Install and run sshx
!curl -sSf https://sshx.io/get | sh

# Function to run the Python application (metube)
def run_python_app():
    print("Starting Python application (metube)...")
    subprocess.run(["pipenv", "run", "python3", "app/main.py"])

# Function to run bore
def run_bore():
    print("Starting bore tunnel...")
    with open("bore_output.log", "w") as log_file:
        subprocess.run(["./bore", "local", "8081", "--to", "bore.pub"], stdout=log_file, stderr=log_file)

# Function to run sshx
def run_sshx():
    print("Starting sshx tunnel...")
    with open("sshx_output.log", "w") as log_file:
        subprocess.run(["sshx"], stdout=log_file, stderr=log_file)

# Function to search for bore.pub URL
def search_bore_url():
    pattern = r'bore\.pub:(\d+)'
    while True:
        try:
            with open("bore_output.log", "r") as log_file:
                content = log_file.read()
                match = re.search(pattern, content)
                if match:
                    port = match.group(1)
                    bore_url = f"http://bore.pub:{port}"
                    print(f"Found bore.pub URL: {bore_url}")
                    return
        except FileNotFoundError:
            pass
        time.sleep(5)

# Function to search for sshx.io URL
def search_sshx_url():
    pattern = r'(https://sshx\.io/s/\w+#\w+)'
    while True:
        try:
            with open("sshx_output.log", "r") as log_file:
                content = log_file.read()
                match = re.search(pattern, content)
                if match:
                    sshx_url = match.group(1)
                    print(f"Found sshx.io URL: {sshx_url}")
                    return
        except FileNotFoundError:
            pass
        time.sleep(5)

# Start metube first
python_app_thread = threading.Thread(target=run_python_app)
python_app_thread.start()

# Wait for a moment to ensure metube is running
time.sleep(2)

# Start bore, sshx, and URL searches in separate threads
bore_thread = threading.Thread(target=run_bore)
sshx_thread = threading.Thread(target=run_sshx)
bore_url_search_thread = threading.Thread(target=search_bore_url)
sshx_url_search_thread = threading.Thread(target=search_sshx_url)

# Start the threads
bore_thread.start()
sshx_thread.start()
bore_url_search_thread.start()
sshx_url_search_thread.start()

# Wait for all threads to finish
python_app_thread.join()
bore_thread.join()
sshx_thread.join()
bore_url_search_thread.join()
sshx_url_search_thread.join()

Unzipping prebuilt UI...
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 10.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package libpython3.11-minimal:amd64.
(Reading database ... 123623 files and directories currently installed.)
Preparing to unpack .../0-libpython3.11-minimal_3.11.10-1+jammy1_amd64.deb ...
Unpacking libpython3.11-minimal:amd64 (3.11.10-1+jammy1) ...
Selecting previously unselected package python3.11-min